In [1]:
import scipy.sparse as sp
import numpy as np
from collections import Counter

In [2]:
from scipy.spatial.distance import euclidean,cosine

In [19]:
a = [0,0,0,0,1,1,1]
b = [0,0,0,0,1,1,0]
c = [1,1,1,1,1,1,1]
d = [0,0,1,1,1,1,0]

In [11]:
def normalize(a):
    return [i/sum(a) for i in a]

In [18]:
print(euclidean(normalize(a),normalize(b)))
print(euclidean(normalize(c),normalize(b)))
print(cosine(normalize(a),normalize(b)))
print(cosine(normalize(c),normalize(b)))

0.408248290463863
0.5976143046671968
0.18350341907227397
0.4654775161751513


In [20]:
print(euclidean(a,b))
print(euclidean(c,b))
print(euclidean(d,b))
print(cosine(a,b))
print(cosine(c,b))

1.0
2.23606797749979
1.4142135623730951
0.18350341907227385
0.4654775161751512


In [23]:
import pysam
import pandas as pd
samfile = pysam.AlignmentFile("/home/miaocj/docker_dir/kNN-overlap-finder/data/real_reads/human/cyclone/aligned.bam", "rb")

In [27]:
def get_bam_stat_df(bam_file):
    bam = pysam.AlignmentFile(bam_file, "rb")
    df_rows = []
    for i, seg in enumerate(bam.fetch()):
        if seg.reference_name == 'chr1_MATERNAL':
            if seg.is_reverse:
                strand = "-"  # 反向链
            else:
                strand = "+"  # 正向链
            if seg.has_tag("NM"):
                mismatch =  seg.get_tag("NM")
            else:
                mismatch = 0 
            row = dict(
                segment_id=i,
                reference_name=seg.reference_name,
                reference_start=seg.reference_start,
                reference_end=seg.reference_end,
                
                mismatch=mismatch,
                query_name=seg.query_name,
                query_alignment_start=seg.query_alignment_start,
                query_length=seg.query_length,
                query_alignment_length=seg.query_alignment_length,
                mapping_quality=seg.mapping_quality,
            )
            df_rows.append(row)
    df = pd.DataFrame(df_rows)
    return df

In [28]:
df = get_bam_stat_df("/home/miaocj/docker_dir/kNN-overlap-finder/data/real_reads/human/cyclone/aligned.bam")

In [74]:
df

,segment_id,reference_name,reference_start,reference_end,mismatch,query_name,query_alignment_start,query_length,query_alignment_length,mapping_quality,percentage
0,0,chr1_MATERNAL,173130702,173134395,300,WTB4000011D2-202307071450251_20230708004102_01...,21,3611,3590,60,0.994184
1,6,chr1_MATERNAL,65629804,65646841,1029,WTB4000011D2-202307071450251_20230708004102_01...,22,16832,16787,60,0.997327
2,13,chr1_MATERNAL,205109248,205111292,139,WTB4000011D2-202307071450251_20230708004102_01...,27,2025,1998,60,0.986667
3,24,chr1_MATERNAL,177432755,177468764,2814,WTB4000011D2-202307071450251_20230708004102_01...,0,35341,35321,60,0.999434
4,27,chr1_MATERNAL,88883631,88884972,78,WTB4000011D2-202307071450251_20230708004102_01...,22,1335,1313,60,0.983521
...,...,...,...,...,...,...,...,...,...,...,...
547298,7208357,chr1_MATERNAL,30202021,30203353,375,WTB420000DAC-202307211437550_20230722070614_00...,0,1288,1288,7,1.000000
547299,7208366,chr1_MATERNAL,2783134,2789676,777,WTB420000DAC-202307211437550_20230722070614_00...,40,6367,6323,60,0.993089
547300,7208373,chr1_MATERNAL,15624991,15649170,995,WTB420000DAC-202307211437550_20230722070614_00...,30,24001,23971,60,0.998750
547301,7208377,chr1_MATERNAL,114966267,114985168,971,WTB420000DAC-202307211437550_20230722070614_00...,1,18637,18610,60,0.998551


In [84]:
##percentage 为reference上的长度占整个querylength的比例
df['percentage'] =(df.reference_end - df.reference_start)/df.query_length
df2 = df.groupby('query_name')['percentage'].max().reset_index()
bad_query = df2[(df2.percentage<0.9) | (df2.percentage>1.2)]['query_name'].tolist()
bad_query_align = df[df.query_name.isin(bad_query)].groupby('query_name').size().reset_index()
bad_query_align[bad_query_align.loc[:,0]>0]

,query_name,0
0,WTB4000011D2-202307071450251_20230707172537_00...,1
1,WTB4000011D2-202307071450251_20230707172537_00...,1
2,WTB4000011D2-202307071450251_20230707172537_00...,1
3,WTB4000011D2-202307071450251_20230707172537_00...,1
4,WTB4000011D2-202307071450251_20230707172537_00...,1
...,...,...
46891,WTB420000DAC-202307211437550_20230723171907_00...,1
46892,WTB420000DAC-202307211437550_20230723171907_00...,1
46893,WTB420000DAC-202307211437550_20230723171907_00...,1
46894,WTB420000DAC-202307211437550_20230723171907_00...,1


In [91]:
##percentage 为比对上的长度占整个querylength的比例
df['percentage'] =df['query_alignment_length']/df['query_length']
max_indices = df.groupby('query_name')['percentage'].idxmax()
result = df.loc[max_indices]

df2 = df.groupby('query_name')['percentage'].max().reset_index()
bad_query = df2[df2.percentage<0.9]['query_name'].tolist()
meta_df = df[ ~df.query_name.isin(bad_query)]

bad_query_align = df[df.query_name.isin(bad_query)].groupby('query_name').size().reset_index()
bad_query_align[bad_query_align.loc[:,0]>0]

,query_name,0
0,WTB4000011D2-202307071450251_20230707172537_00...,1
1,WTB4000011D2-202307071450251_20230707172537_00...,1
2,WTB4000011D2-202307071450251_20230707172537_00...,1
3,WTB4000011D2-202307071450251_20230707172537_00...,1
4,WTB4000011D2-202307071450251_20230707172537_00...,1
...,...,...
46682,WTB420000DAC-202307211437550_20230723171907_00...,1
46683,WTB420000DAC-202307211437550_20230723171907_00...,1
46684,WTB420000DAC-202307211437550_20230723171907_00...,1
46685,WTB420000DAC-202307211437550_20230723171907_00...,1


In [93]:
len(set(result['query_name'].tolist()))

501541

In [100]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from snakemake_stub import *
from dataclasses import dataclass, field
import collections
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
import seaborn
import sharedmem
import csv
import sys
from intervaltree import Interval, IntervalTree
from collections import defaultdict
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/scripts")

from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges

In [101]:
def get_bam_stat_df(bam_file):
    bam = pysam.AlignmentFile(bam_file, "rb")
    df_rows = []
    for i, seg in enumerate(bam.fetch()):
        if (seg.reference_name == 'chr6_MATERNAL') and (seg.reference_start > 28476949) and (seg.reference_end < 34231258):
            if seg.is_reverse:
                strand = "-"  # 反向链
            else:
                strand = "+"  # 正向链
            if seg.has_tag("NM"):
                mismatch =  seg.get_tag("NM")
            else:
                mismatch = 0 
            row = dict(
                segment_id=i,
                reference_name=seg.reference_name,
                reference_start=seg.reference_start,
                reference_end=seg.reference_end,
                reference_strand = strand,
                mismatch=mismatch,
                query_name=seg.query_name,
                query_alignment_start=seg.query_alignment_start,
                query_length=seg.query_length,
                query_alignment_length=seg.query_alignment_length,
                mapping_quality=seg.mapping_quality,
            )
            df_rows.append(row)
    df = pd.DataFrame(df_rows)
    return df



df = get_bam_stat_df("/home/miaocj/docker_dir/kNN-overlap-finder/data/real_reads/human/cyclone/aligned.bam")
df['percentage'] =df['query_alignment_length']/df['query_length']
max_indices = df.groupby('query_name')['percentage'].idxmax()
result = df.loc[max_indices]
meta_df = result[result.percentage>0.9]
all_reads_num = len(result['query_name'].tolist())

bad_query = result[result.percentage<=0.9].tolist()
bad_query_align = df[df.query_name.isin(bad_query)].groupby('query_name').size().reset_index()
bad_query_num = bad_query_align.shape[0]
bad_in_all = bad_query_num/all_reads_num
lines_number = bad_query_align[bad_query_align.loc[:,0]>1].shape[0]
per = lines_number/bad_query_num
print('There are %d(%f) reads do not have >90% percentage and %d(%f) of them have more than one alignment.'%(bad_query_num,bad_in_all,lines_number,per))
  
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

fig, ax = plt.subplots(figsize=(8, 6))

# 调用 plot_read_graph 函数
plot_read_graph(
    ax=ax,
    metadata=meta_df,        # 替换为你的 metadata
    query_graph=reference_graph,  # 替换为你的 reference_graph
    layout_method="sfdp",     # 布局方法
    node_size=3,              # 节点大小
    seed=43                   # 随机种子
)

fig.savefig("output_graph.png", dpi=300, bbox_inches="tight")
  

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [ ]:

def build_interval_trees(read_dict):
    """
    将每条 read (保留所有比对位点) 根据 reference_name 构建 IntervalTree。
    返回: { ref_name: IntervalTree(...) }
    """
    interval_trees = defaultdict(IntervalTree)
    
    for rname, alignments in read_dict.items():
        for info in alignments:
            ref = info["ref_name"]
            start = info["start"]
            end = info["end"]
            
            # Interval 的 data 存储 read_name；后续再从 read_dict 获取其他信息
            interval_trees[ref].add(Interval(start, end, data=rname))
        
    return interval_trees

In [35]:
len(set(df.loc[:,'query_name'].tolist()))

501541

In [34]:
len(set(df[df.percentage > 0.9].loc[:,'query_name'].tolist()))

454789

In [5]:
neighbors = np.load('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/metagenome/bacteria/pbsim_ONT_95_20k/kmer_16/MinHash_Jaccard_None_None_nbr_matrix.npz')

In [11]:
nbr = neighbors['arr_0']

In [17]:
Counter(nbr[:,0].tolist())

Counter({-1: 118284,
         849: 16,
         848: 15,
         672: 12,
         673: 12,
         946: 11,
         947: 10,
         880: 9,
         919: 9,
         333: 8,
         995: 8,
         769: 8,
         768: 8,
         732: 8,
         881: 7,
         924: 7,
         332: 7,
         602: 7,
         619: 6,
         618: 6,
         797: 6,
         763: 6,
         774: 6,
         903: 6,
         730: 6,
         685: 6,
         974: 6,
         603: 5,
         888: 5,
         444: 5,
         411: 5,
         918: 5,
         910: 5,
         907: 5,
         994: 5,
         270: 5,
         677: 5,
         722: 5,
         567: 5,
         861: 5,
         952: 5,
         57: 5,
         676: 4,
         959: 4,
         802: 4,
         902: 4,
         271: 4,
         445: 4,
         513: 4,
         297: 4,
         644: 4,
         922: 4,
         294: 4,
         295: 4,
         775: 4,
         41: 4,
         847: 4,
         700: 4,
      